#### 상대모멘텀
- 10개의 종목을 이용하여 투자 전략 시스템

    1. 월초부터 월말까지의 수정종가를 이용하여 수익율 생성
    2. 월별 수익율이 높은 n개를 선택
    3. 해당하는 종목을 매수하여 해당하는 종목에 신호가 없어지면 매도하는 형식으로 수익율 생성

In [1]:
import pandas as pd
import numpy as np
import os
from glob import glob
import warnings

In [2]:
# warning 메시지 출력 방지
warnings.filterwarnings('ignore')

In [3]:
# 웗별 수익율을 계산하는 함수
def create_1m_rtn(_df,
                  _ticker,
                  _start = '2010-01-01',
                  _col = 'Adj Close'):
    # 복사본 생성
    result = _df.copy()
    # Date 컬럼이 존재하면 index로 변경
    if 'Date' in result.columns:
        result.set_index('Date', inplace=True)
    # index를 시계열로 변경
    result.index = pd.to_datetime(result.index)
    # 시작 시간과 기준이 되는 컬럼으로 필터링
    result = result.loc[_start:, [_col]]
    # 기준 년월 컬럼을 생성
    result['STD-YM'] = result.index.strftime('%Y-%m')
    result['1n-rtn'] = 0
    result['CODE'] = _ticker
    # 기준년월(STD-YM)의 유니크한 리스트를 생성
    ym_list = result['STD-YM'].unique()
    return result, ym_list

In [4]:
aapl = pd.read_csv("../../csv/AAPL.csv")

In [5]:
sample_aapl, ym_list = create_1m_rtn(aapl, 'AAPL')

In [6]:
sample_aapl

,Adj Close,STD-YM,1n-rtn,CODE
Date,,,,
2010-01-04,26.782711,2010-01,0,AAPL
2010-01-05,26.829010,2010-01,0,AAPL
2010-01-06,26.402260,2010-01,0,AAPL
2010-01-07,26.353460,2010-01,0,AAPL
2010-01-08,26.528664,2010-01,0,AAPL
...,...,...,...,...
2019-06-18,198.449997,2019-06,0,AAPL
2019-06-19,197.869995,2019-06,0,AAPL
2019-06-20,199.460007,2019-06,0,AAPL


In [7]:
ym_list

array(['2010-01', '2010-02', '2010-03', '2010-04', '2010-05', '2010-06',
       '2010-07', '2010-08', '2010-09', '2010-10', '2010-11', '2010-12',
       '2011-01', '2011-02', '2011-03', '2011-04', '2011-05', '2011-06',
       '2011-07', '2011-08', '2011-09', '2011-10', '2011-11', '2011-12',
       '2012-01', '2012-02', '2012-03', '2012-04', '2012-05', '2012-06',
       '2012-07', '2012-08', '2012-09', '2012-10', '2012-11', '2012-12',
       '2013-01', '2013-02', '2013-03', '2013-04', '2013-05', '2013-06',
       '2013-07', '2013-08', '2013-09', '2013-10', '2013-11', '2013-12',
       '2014-01', '2014-02', '2014-03', '2014-04', '2014-05', '2014-06',
       '2014-07', '2014-08', '2014-09', '2014-10', '2014-11', '2014-12',
       '2015-01', '2015-02', '2015-03', '2015-04', '2015-05', '2015-06',
       '2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12',
       '2016-01', '2016-02', '2016-03', '2016-04', '2016-05', '2016-06',
       '2016-07', '2016-08', '2016-09', '2016-10', 

In [8]:
# listdir() 파일의 이름의 목록
os.listdir('./data')
# glob() 파일의 경로와 이름의 목록
files = glob('./data/*.csv')

In [9]:
files

['./data/GDX.csv',
 './data/AMZN.csv',
 './data/GM.csv',
 './data/MSFT.csv',
 './data/USM.csv',
 './data/GLD.csv',
 './data/SPY.csv',
 './data/BND.csv',
 './data/AAPL.csv',
 './data/SLV.csv']

In [10]:
# 종목별 데이터프레임 생성

# 월말의 데이터를 모아주는 작업
# 월별 수익률 계산
stock_df = pd.DataFrame()
month_last_df = pd.DataFrame()

for file in files:
    # file : 주식데이터의 경로와 파일명
    # 경로와 파일명을 나눠준다.
    folder, name = os.path.split(file)
    # name : AAPL.csv --> AAPL  .csv 나눈다.
    head, tail = os.path.splitext(name)
    # head 변수는 create_1m_rtn()함수에서 _ticker 사용

    # csv 파일을 로드
    read_df = pd.read_csv(file)

    # create_1m_rtn() 함수를 호출
    price_df, ym_list = create_1m_rtn(read_df, _ticker = head)

    # price_df를 stock_df에 단순 행결합 모두 결합
    stock_df = pd.concat( [stock_df, price_df] )

    # 두번째 반복문 생성
    # price_df의 월별 수익율 계산하는 반복문
    for ym in ym_list:
        # ym : 기준년월
        flag = price_df['STD-YM'] == ym
        # 월초 수정 종가
        buy = price_df.loc[flag].iloc[0, 0]
        # 월말 수정 종가
        sell = price_df.loc[flag].iloc[-1, 0]
        # 수익률 계산
        m_rtn = sell / buy
        # price_df에 flag 조건과 1m_rtn 컬럼에 대입
        price_df.loc[flag, '1m_rtn'] = m_rtn
        # 월말의 데이터를 month_last_df에 단순 행 결합
        last_data = price_df.loc[flag, ['CODE', '1m_rtn']].tail(1)
        month_last_df = pd.concat( [month_last_df, last_data] )

In [11]:
month_last_df

,CODE,1m_rtn
Date,,
2010-01-29,GDX,0.853490
2010-02-26,GDX,1.022124
2010-03-31,GDX,0.994848
2010-04-30,GDX,1.088577
2010-05-28,GDX,0.999599
...,...,...
2019-02-28,SLV,0.980563
2019-03-29,SLV,0.998592
2019-04-30,SLV,0.990813


In [12]:
# 데이터 백업
month_rtn_df = month_last_df.copy()

In [13]:
# 인덱스를 리셋
month_rtn_df.reset_index(inplace=True)

In [14]:
# pivot_table을 이용하요 테이블을 재구조화
month_rtn_df = month_rtn_df.pivot_table(
    index = 'Date',
    columns = 'CODE',
    values = '1m_rtn'
)

In [15]:
month_rtn_df = month_rtn_df.rank(
    axis=1,
    ascending=False,
    method='max',
    pct=True
)

In [16]:
# 상위의 15% 종목만 선택
# where( 조건식, 조건식이 거짓인 경우 변경할 데이터 값 )
month_rtn_df = month_rtn_df.where(month_rtn_df < 0.15, 0)

In [17]:
# 0이 아닌 데이터는 1로 변환
month_rtn_df[month_rtn_df != 0] = 1

In [18]:
month_rtn_df

CODE,AAPL,AMZN,BND,GDX,GLD,GM,MSFT,SLV,SPY,USM
Date,,,,,,,,,,
2010-01-29,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-02-26,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-03-31,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-04-30,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-05-28,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2019-02-28,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2019-03-29,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-04-30,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [37]:
# CODE 값들로 이루어진 리스트 생성
stock_codes = stock_df['CODE'].unique()
# month_rtn_df.columns

In [38]:
month_rtn_df.loc['2010-01-29', month_rtn_df.loc['2010-01-29'] == 1].index

Index(['BND'], dtype='object', name='CODE')

In [39]:
sig_dict = dict()

for date in month_rtn_df.index:
    # date : 인덱스의 값들
    ticker_list = list(
        month_rtn_df.loc[date, month_rtn_df.loc[date] == 1].index
    )
    sig_dict[date] = ticker_list

In [40]:
sig_dict

{Timestamp('2010-01-29 00:00:00'): ['BND'],
 Timestamp('2010-02-26 00:00:00'): ['AAPL'],
 Timestamp('2010-03-31 00:00:00'): ['AAPL'],
 Timestamp('2010-04-30 00:00:00'): ['AAPL'],
 Timestamp('2010-05-28 00:00:00'): ['GLD'],
 Timestamp('2010-06-30 00:00:00'): ['GDX'],
 Timestamp('2010-07-30 00:00:00'): ['USM'],
 Timestamp('2010-08-31 00:00:00'): ['GDX'],
 Timestamp('2010-09-30 00:00:00'): ['AMZN'],
 Timestamp('2010-10-29 00:00:00'): ['SLV'],
 Timestamp('2010-11-30 00:00:00'): ['SLV'],
 Timestamp('2010-12-31 00:00:00'): ['SLV'],
 Timestamp('2011-01-31 00:00:00'): ['AAPL'],
 Timestamp('2011-02-28 00:00:00'): ['SLV'],
 Timestamp('2011-03-31 00:00:00'): ['SLV'],
 Timestamp('2011-04-29 00:00:00'): ['SLV'],
 Timestamp('2011-05-31 00:00:00'): ['BND'],
 Timestamp('2011-06-30 00:00:00'): ['MSFT'],
 Timestamp('2011-07-29 00:00:00'): ['SLV'],
 Timestamp('2011-08-31 00:00:00'): ['GLD'],
 Timestamp('2011-09-30 00:00:00'): ['AMZN'],
 Timestamp('2011-10-31 00:00:00'): ['GM'],
 Timestamp('2011-11-30 00:

In [41]:
stock_df.head(2)

,Adj Close,STD-YM,1n-rtn,CODE
Date,,,,
2010-01-04,44.908779,2010-01,0,GDX
2010-01-05,45.341774,2010-01,0,GDX


In [42]:
# 거래내역 컬럼을 생성
def create_trade_book(_df, _code):
    # 복사본 생성
    book = _df[_code].copy()
    book['STD-YM'] = book.index.strftime('%Y-%m')
    for c in _code:
        book[f"p_{c}"] = ''
        book[f"r_{c}"] = ''
    return book

In [43]:
# stock_df를 재구조화
stock_c_matrix = stock_df.reset_index().pivot_table(
    index = 'Date',
    columns = 'CODE',
    values = stock_df.columns[0]
)

In [44]:
stock_c_matrix

CODE,AAPL,AMZN,BND,GDX,GLD,GM,MSFT,SLV,SPY,USM
Date,,,,,,,,,,
2010-01-04,26.782711,133.899994,60.611969,44.908779,109.800003,NaN,24.525019,17.230000,93.675278,36.015179
2010-01-05,26.829010,134.690002,60.789135,45.341774,109.699997,NaN,24.532942,17.510000,93.923241,35.998024
2010-01-06,26.402260,132.250000,60.766037,46.443077,111.510002,NaN,24.382378,17.860001,93.989357,35.680672
2010-01-07,26.353460,130.000000,60.719822,46.217175,110.820000,NaN,24.128809,17.889999,94.386139,35.208931
2010-01-08,26.528664,133.520004,60.781410,46.913723,111.370003,NaN,24.295214,18.150000,94.700218,34.651424
...,...,...,...,...,...,...,...,...,...,...
2019-06-18,198.449997,1901.369995,82.397118,23.670000,127.120003,36.700001,135.160004,14.050000,290.984741,50.540001
2019-06-19,197.869995,1908.790039,82.676468,24.000000,127.889999,36.779999,135.690002,14.170000,291.641541,50.040001
2019-06-20,199.460007,1918.189941,82.806168,25.049999,131.110001,36.959999,136.949997,14.450000,294.427979,49.320000


In [45]:
book = create_trade_book(stock_c_matrix, stock_codes)

In [47]:
book.columns

Index(['GDX', 'AMZN', 'GM', 'MSFT', 'USM', 'GLD', 'SPY', 'BND', 'AAPL', 'SLV',
       'STD-YM', 'p_GDX', 'r_GDX', 'p_AMZN', 'r_AMZN', 'p_GM', 'r_GM',
       'p_MSFT', 'r_MSFT', 'p_USM', 'r_USM', 'p_GLD', 'r_GLD', 'p_SPY',
       'r_SPY', 'p_BND', 'r_BND', 'p_AAPL', 'r_AAPL', 'p_SLV', 'r_SLV'],
      dtype='object', name='CODE')

In [48]:
book.head(2)

CODE,GDX,AMZN,GM,MSFT,USM,GLD,SPY,BND,AAPL,SLV,...,p_GLD,r_GLD,p_SPY,r_SPY,p_BND,r_BND,p_AAPL,r_AAPL,p_SLV,r_SLV
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,44.908779,133.899994,NaN,24.525019,36.015179,109.800003,93.675278,60.611969,26.782711,17.23,...,,,,,,,,,,
2010-01-05,45.341774,134.690002,NaN,24.532942,35.998024,109.699997,93.923241,60.789135,26.829010,17.51,...,,,,,,,,,,


In [51]:
sig_dict.items()

dict_items([(Timestamp('2010-01-29 00:00:00'), ['BND']), (Timestamp('2010-02-26 00:00:00'), ['AAPL']), (Timestamp('2010-03-31 00:00:00'), ['AAPL']), (Timestamp('2010-04-30 00:00:00'), ['AAPL']), (Timestamp('2010-05-28 00:00:00'), ['GLD']), (Timestamp('2010-06-30 00:00:00'), ['GDX']), (Timestamp('2010-07-30 00:00:00'), ['USM']), (Timestamp('2010-08-31 00:00:00'), ['GDX']), (Timestamp('2010-09-30 00:00:00'), ['AMZN']), (Timestamp('2010-10-29 00:00:00'), ['SLV']), (Timestamp('2010-11-30 00:00:00'), ['SLV']), (Timestamp('2010-12-31 00:00:00'), ['SLV']), (Timestamp('2011-01-31 00:00:00'), ['AAPL']), (Timestamp('2011-02-28 00:00:00'), ['SLV']), (Timestamp('2011-03-31 00:00:00'), ['SLV']), (Timestamp('2011-04-29 00:00:00'), ['SLV']), (Timestamp('2011-05-31 00:00:00'), ['BND']), (Timestamp('2011-06-30 00:00:00'), ['MSFT']), (Timestamp('2011-07-29 00:00:00'), ['SLV']), (Timestamp('2011-08-31 00:00:00'), ['GLD']), (Timestamp('2011-09-30 00:00:00'), ['AMZN']), (Timestamp('2011-10-31 00:00:00'), [

In [53]:
for date, values in sig_dict.items():
    # print(date)
    # print(values)
    for stock in values:
        # stock : 종목명
        book.loc[date, f"p_{stock}"] = f"ready_{stock}"

In [61]:
book['p_AMZN'].value_counts()

p_AMZN
              2360
ready_AMZN      24
Name: count, dtype: int64

In [ ]:
# 거래내역 추가
def trading(_book, s_codes):
    std_ym = ''
    buy_phase = False

    # 종목별로 순회 (반복)
    for code in s_codes:
        # 인덱스별 반복
        for idx in _book.index:
            # 특정 종목 코드에서 포지션을 잡는다.
            if (_book.loc[idx, f"p_{code}"] == "") & \
                (_book.shift().loc[idx, f"p_{code}"] == f"ready_{code}"):
                std_ym = _book.loc[idx, 'STD-YM']
                buy_phase = True
            # 해당 종목 코드에서 신호가 잡혀있다면 매수 유지
            if (_book.loc[idx, f"p_{code}"] == '') & \
                (_book.loc[idx, "STD-YM"] == std_ym) & \
                (buy_phase):
                _book.loc[idx, f"r_{code}"] = f"buy_{code}"
            # buy_phase, std_ym 초기화
            if _book.loc[idx, f"p_{code}"] == '':
                std_ym = ''
                buy_phase = False
    return _book
